## prepare for compact result

In [217]:
import pandas as pd
from pandas import read_csv
import numpy as np
import types
import csv
detail = pd.read_csv("RegularSeasonDetailedResults.csv");
regular=pd.read_csv('RegularSeasonCompactResults.csv');
regular=regular[regular['Season']>2002];
regular=regular[regular['Season']<2017];
detail=detail[detail['Season']<2017]
#df['elderly'] = np.where(df['age']>=50, 'yes', 'no')

In [218]:
regular['Season'].shape
#regular.head();
#regular.shape;

(71241,)

In [219]:
# add result into regular and count win probability for each team
regular = regular.rename(columns={"Wteam":"team1", "Lteam":"team2"});
regular = regular.rename(columns={"Wscore":"team1_score", "Lscore":"team2_score"})
regular["team1_result"] = np.repeat(1,71241);
regular["team2_result"]=np.repeat(0,71241);
regular.head()

,Season,Daynum,team1,team1_score,team2,team2_score,Wloc,Numot,team1_result,team2_result
74048,2003,10,1104,68,1328,62,N,0,1,0
74049,2003,10,1272,70,1393,63,N,0,1,0
74050,2003,11,1266,73,1437,61,N,0,1,0
74051,2003,11,1296,56,1457,50,N,0,1,0
74052,2003,11,1400,77,1208,71,N,0,1,0


In [221]:
regular_team1=regular[regular.columns[[0,1,2,3,7]] ];
team1 = regular_team1.rename(columns={"team1":'team'});
team1 = team1.rename(columns={"team1_score":'score'});
regular_team2=regular[regular.columns[[0,1,4,5,7]] ];
team2 = regular_team2.rename(columns={"team2":'team'});
team2 = team2.rename(columns={"team2_score":'score'});
teams = [team1, team2]
team = pd.concat(teams);
score=team.groupby(['Season','team'])['score'].mean();
Numot=team.groupby(['Season','team'])['Numot'].mean();
score=score.to_frame().reset_index();
Numot=Numot.to_frame().reset_index();
score=score.merge(Numot,on=['Season','team']);
score.head()

,Season,team,score,Numot
0,2003,1102,57.250000,0.000000
1,2003,1103,78.777778,0.296296
2,2003,1104,69.285714,0.035714
3,2003,1105,71.769231,0.153846
4,2003,1106,63.607143,0.035714


In [222]:
regular_team1_getaveragescore=regular.groupby(['team1','Season'])['team1_score'].mean();#win get score
regular_team2_loseaveragescore=regular.groupby(['team2','Season'])['team2_score'].mean();# lose get score
regular_team1_LOSEaveragescore=regular.groupby(['team1','Season'])['team2_score'].mean();# win lose score
regular_team2_GETaveragescore=regular.groupby(['team2','Season'])['team1_score'].mean();# lose lose score
regular_win_get = regular_team1_getaveragescore.to_frame().reset_index();
regular_lose_get = regular_team2_loseaveragescore.to_frame().reset_index();
regular_win_lose = regular_team1_LOSEaveragescore.to_frame().reset_index();
regular_lose_lose = regular_team2_GETaveragescore.to_frame().reset_index();
regular_lose_get = regular_lose_get.rename(columns={"team2":'team1'});
regular_get=regular_win_get.merge(regular_lose_get,on=['Season','team1'])
regular_get = regular_get.rename(columns={"team1_score":'win_get_score'});
regular_get = regular_get.rename(columns={"team2_score":'lose_get_score'});

regular_lose_lose.head()
regular_lose_lose = regular_lose_lose.rename(columns={"team2":'team1'});
regular_lose=regular_win_lose.merge(regular_lose_lose,on=['Season','team1'])
regular_lose = regular_lose.rename(columns={"team1_score":'win_lose_score'});
regular_lose = regular_lose.rename(columns={"team2_score":'lose_lose_score'});
regular_score=regular_get.merge(regular_lose, on=['Season','team1']);
regular_score=regular_score.rename(columns={"team1":"team"})

In [223]:
regular_team_allscore=regular_score.merge(score,on=['Season','team']);

In [224]:
win=regular.groupby(['team1','Season'])['team1_result'].size();
win=win.to_frame().reset_index();
win = win.rename(columns={"team1":'team'});
games_win= regular.groupby(['Season','team1']).size();
games_win=games_win.to_frame().reset_index();
games_lose= regular.groupby(['Season','team2']).size();
games_lose=games_lose.to_frame().reset_index();
games_win = games_win.rename(columns={"team1":'team'});
games_lose = games_lose.rename(columns={"team2":'team'});
game = [games_win, games_lose]
game = pd.concat(game);
game=game.rename(columns={0:'number'})
number=game.groupby(['Season','team'])['number'].sum();
number=number.to_frame().reset_index();
number=number.merge(win,on=['team','Season']);
number['rate']=number['team1_result']/number['number'].values;
regular_score_winrate=number.merge(regular_team_allscore,on=['Season','team']);
regular_score_winrate=regular_score_winrate.rename(columns={'team1_result':'win_game'});
regular_score_winrate=regular_score_winrate.rename(columns={'number':'game'});


In [225]:
regular_score_winrate.to_csv('regular_score.csv')

In [228]:
regular_score_winrate.head()

,Season,team,game,win_game,rate,win_get_score,lose_get_score,lose_lose_score,win_lose_score,score,Numot
0,2003,1102,28,12,0.428571,68.750000,48.625000,53.166667,59.875000,57.250000,0.000000
1,2003,1103,27,13,0.481481,87.769231,70.428571,78.384615,77.928571,78.777778,0.296296
2,2003,1104,28,17,0.607143,74.705882,60.909091,61.529412,70.363636,69.285714,0.035714
3,2003,1105,26,7,0.269231,79.428571,68.947368,66.428571,80.421053,71.769231,0.153846
4,2003,1106,28,13,0.464286,68.307692,59.533333,57.923077,68.800000,63.607143,0.035714
